# Text 2: Latent semantic indexing
**Internet Analytics - Lab 4**

---

**Group:** R

**Names:**

* Raphael Strebel
* Raphaël Barman
* Thierry Bossy

---

#### Instructions

*This is a template for part 2 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [1]:
import pickle
import numpy as np
from scipy.sparse.linalg import svds

id2name = np.load('id2name.npy').item()
name2id = np.load('name2id.npy').item()
idx2Term = np.load('idx2Term.npy').item()
term2Idx = np.load('term2Idx.npy').item()
idx2Course = np.load('idx2Course.npy').item()
course2Idx = np.load('course2Idx.npy').item()
numTerms = len(term2Idx)
numCourses = len(course2Idx)

idf = np.load('idf.npy')

X = np.load('X.npy').item()

## Exercise 4.4: Latent semantic indexing

In [103]:
k = 300
U, S, Vt = svds(X,k)

- S is the scaling of the matrix, thus it represents the importance of the topic
- U in SVD represents the row space, so in our case it relates the terms to the topics
- Vt is the column space, so it relates the course to the topics

In [104]:
print("The top-20 eigenvalues of X : ")
list(S[-20:][::-1])

The top-20 eigenvalues of X : 


[39.419623854044232,
 25.923089457921758,
 24.210111699354407,
 22.594426256236058,
 21.831873950133076,
 21.682666730617587,
 20.954186974052032,
 20.347456189532462,
 20.172810241138365,
 19.455031816896174,
 19.001939163825636,
 18.815930051692394,
 18.752118551849126,
 18.445992548658776,
 18.009541190364775,
 17.848554284297016,
 17.732130297409917,
 17.540732006707788,
 17.44874764370509,
 17.410762340181716]

## Exercise 4.5: Topic extraction

In [105]:
for i in reversed(range(k-10,k)):
    print('Topic %d top 10 terms:'%(k-i))
    topTerms = np.argsort(-np.abs(U.T[i]))[:10]
    for term in topTerms:
        print('  - %s\t%.3f'%(idx2Term[term],U.T[i][term]))
    print('Topic %d top 10 courses:'%(k-i))
    topCourses = np.argsort(-np.abs(Vt[i]))[:10]
    for course in topCourses:
        print('  - %s\t%.3f'%(id2name[idx2Course[course]],Vt[i][course]))

Topic 1 top 10 terms:
  - problem	0.099
  - report	0.094
  - research	0.092
  - presentation	0.091
  - plan	0.089
  - material	0.085
  - process	0.083
  - scientific	0.078
  - engineering	0.078
  - work	0.077
Topic 1 top 10 courses:
  - Lab immersion III	0.131
  - Lab immersion I	0.128
  - Bioprocesses and downstream processing	0.123
  - Renewable energy and solar architecture in Davos	0.116
  - Lab immersion II	0.113
  - Nanobiotechnology and biophysics	0.106
  - Semester project in Bioengineering	0.106
  - General physics (English) II	0.093
  - Principles of finance	0.089
  - Project in bioengineering and biosciences	0.087
Topic 2 top 10 terms:
  - wetlab	0.221
  - experimentation	0.205
  - obtained	0.173
  - experiment	0.157
  - laboratory-based	0.144
  - ou	0.126
  - semaines	0.126
  - plan	0.114
  - research	0.105
  - neuroscience	0.104
Topic 2 top 10 courses:
  - Lab immersion III	0.408
  - Lab immersion I	0.392
  - Lab immersion II	0.355
  - Semester project in Bioengineering	0.

1. Research
2. Laboratory
3. Finances
4. Architecture
5. Bio economy 
6. Microscopy
7. Life science
8. Micro technology
9. Biomechanics
10. Cultural Heritage

## Exercise 4.6: Document similarity search in concept-space

In [131]:
U[0].shape

(300,)

In [130]:
def sim(Ut,d):
    Sig = np.diag(S)
    Vd = Vt.T[d]
    SVd = np.dot(Sig,Vd)
    return np.dot(Ut,np.dot(Sig,Vd.T))/(np.sqrt(np.dot(Ut,Ut))*np.sqrt(np.dot(SVd,SVd)))

def query(terms):
    terms = terms.split(' ')
    
    Ut = U[term2Idx[terms[0]]]
    for term in terms[1:]:
        Ut += U[term2Idx[term]]
    results = np.array([sim(Ut,d) for d in range(numCourses)])
    top = np.argsort(-results)[:5]
    # We filter all courses with null scores
    top = [course for course in top if results[course] != 0.0]
    
    for course in top:
        print('   -',id2name[idx2Course[course]],'\t%.3f'%results[course])

In [132]:
print('Top courses for vsm:')
print("""   - Computational Social Media 	0.186""")
print('Top courses for lsi:')
query('facebook')

Top courses for vsm:
   - Computational Social Media 	0.186
Top courses for lsi:
   - Computational Social Media 	0.918
   - Social media 	0.680
   - How people learn I 	0.434
   - How people learn II 	0.352
   - Privacy Protection 	0.297


We have more courses suggestion than previously. This is due to the dimensionality reduction, before the exact term 'Facebook' only appeared in Computational Social Media.

However now, some other terms have been "assimilated" in the same topic as 'facebook', thus giving us more results. The course Computational Social Media still is at the top, since it has the term facebook.

In [135]:
print('Top courses for vsm:')
print("""   - Applied probability & stochastic processes 	0.568
   - Applied stochastic processes 	0.566
   - Markov chains and algorithmic applications 	0.358
   - Supply chain management 	0.357
   - Statistical Sequence Processing 	0.299""")
print('Top courses for lsi:')
query('markov chain')

Top courses for vsm:
   - Applied probability & stochastic processes 	0.568
   - Applied stochastic processes 	0.566
   - Markov chains and algorithmic applications 	0.358
   - Supply chain management 	0.357
   - Statistical Sequence Processing 	0.299
Top courses for lsi:
   - Supply chain management 	0.840
   - Mathematical models in supply chain management 	0.777
   - Applied probability & stochastic processes 	0.610
   - Markov chains and algorithmic applications 	0.587
   - Applied stochastic processes 	0.578


Since the similarity formula tries to 'reconstruct' the term/document value of the intial matrix $X$, but it approximates it with only 300 topics.

Thus it is likely that 'markov' has been associated with 'chain', and thus has less weight than previously, resulting in having the course with an high tf-idf for 'chain' being the most relevant to the query.

## Exercise 4.7: Document-document similarity

We can simply to a cosine similarity between $S\mathbf{v}^T_{d1}$ and $S\mathbf{v}^T_{d2}$.
The idea is to compare the two documents to topic similarity, but to also weight it by the importance of the topic.

For example, let's say we have 3 documents $d_{1,2,3}$, with $k$ topics $t_{1,2,..k}$ with the importance of $t_1$ > $t_2$ > $t_k$ and that $d_1$ and $d_2$ have a topic $t_i$ very similar and all other not similar at all and that $d_1$ and $d_3$ have a topic $t_j$ very similar and also other topics not similar (and that $t_i$ and $t_j$ are not similar for $d_2 and $d_3$).

Then it makes more sense that $d_1$ is rated as more similar to $d_2$ than to $d_3$, since the $t_1$ is more important thatn $t_2$.

In [111]:
def docSim(d1,d2):
    Sig = np.diag(S)
    SVtd1 = np.dot(Sig,Vt[:,d1])
    SVtd2 = np.dot(Sig,Vt[:,d2])
    return np.dot(SVtd1,SVtd2)/(np.sqrt(np.dot(SVtd1,SVtd1))*np.sqrt(np.dot(SVtd2,SVtd2)))

In [112]:
com308 = course2Idx['COM-308']
# All courses without com 308
coursesFiltered = [x for x in idx2Course.keys() if not x == com308]
top = np.argsort(-np.array(list(map(lambda d: docSim(com308,d), coursesFiltered))))[:5]
print("Top 5 classes most similar to com-308 :")
list(map(lambda x: id2name[idx2Course[x]], top))

Top 5 classes most similar to com-308 :


['Martingales in financial mathematics',
 'Risk management',
 'D. Thinking: real problems, human-focused solutions',
 'Neurosciences II : cellular mechanisms of brain function',
 'Working group in Topology II']